In [1]:
import json
import geopandas as gpd
import pandas as pd
import math
from shapely.geometry import Point, MultiPolygon

import csv
import numpy as np
import random
import matplotlib.pyplot as plt

import os


In [3]:
geoDF = gpd.read_file("../city.geojson")
geoDF['wardNo'] = geoDF['wardNo'].astype(int)
geoDF['wardIndex'] = geoDF['wardNo'] - 1
geoDF = geoDF[['wardIndex','wardNo', 'wardName', 'geometry']]
geoDF['wardBounds'] = geoDF.apply(lambda row: MultiPolygon(row['geometry']).bounds, axis=1)
geoDF.sort_values('wardIndex', inplace=True)

In [4]:
demographics = pd.read_csv("../demographics.csv")
for i in range(geoDF.shape[0]):
    if geoDF["wardName"].iloc[i] != demographics['wardName'].iloc[i]:
        print(f"WARNING: Check if this is a mismatch!")
        print(f"{i}\t city.geojson: {geoDF['wardName'].iloc[i]}\t demographics: {demographics['wardName'].iloc[i]}")    

14	 city.geojson: Chembur	 demographics: Chembur (East)


In [4]:
def sampleRandomLatLong(wardIndex):
    #IMPORTANT: geoDF uses (lon, lat) order
    (lon1,lat1,lon2,lat2) = geoDF['wardBounds'][wardIndex]
    while True:
        lat = random.uniform(lat1,lat2)
        lon = random.uniform(lon1,lon2)
        point = Point(lon,lat)
        if MultiPolygon(geoDF['geometry'][wardIndex]).contains(point):
            return (lat,lon)

In [5]:
geoDFslums = gpd.read_file('slumClusters.geojson')
wardslums = [[] for _ in range(len(geoDF))]
for i in range(len(geoDFslums)):
    for j in range(len(geoDF)):
        if geoDFslums["geometry"][i].intersects(geoDF["geometry"][j]):
            wardslums[j].append(i)

In [10]:
numpoints = 1000
nwards = 24
for i in range(nwards):
    fname=str(i)+".csv"
    with open(fname, mode='w+') as file:
        writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        for j in range(numpoints):
            (lat,lon) = sampleRandomLatLong(i)
            writer.writerow([str(lat),str(lon)])

In [13]:
numpoints = 1000
for wardIndex in range(nwards):
    print(f"Generating points for ward {wardIndex+1}")
    
    (lon1,lat1,lon2,lat2) = geoDF['wardBounds'][wardIndex]
    
    fname = f"{wardIndex}_hd.csv"
    with open(fname, mode='w+') as file:
        writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

        count = 0
        while(count < numpoints):
            (lat,lon) = sampleRandomLatLong(wardIndex)
            point = Point(lon,lat) #IMPORTANT: Point takes in order of longitude, latitude

            found=False
            if len(wardslums[wardIndex])==0:
                found=True
            else:
                for i in wardslums[wardIndex]:
                    if geoDFslums["geometry"][i].contains(point):
                        found=True
                        break
            if(found):
                writer.writerow([str(lat),str(lon)])
                count+=1
                if count % 20 == 0:
                    print(".",end='')
        print("done")

Generating points for ward 1
..................................................done
Generating points for ward 2
..................................................done
Generating points for ward 3
..................................................done
Generating points for ward 4
..................................................done
Generating points for ward 5
..................................................done
Generating points for ward 6
..................................................done
Generating points for ward 7
..................................................done
Generating points for ward 8
..................................................done
Generating points for ward 9
..................................................done
Generating points for ward 10
..................................................done
Generating points for ward 11
..................................................done
Generating points for ward 12
............................................